# This notebook is dedicated to trying to add a derivation for the bias vector I made by hand, and verifying that it is correct with the other methods

# SInce I am using bias vectors, I won'y be verifying by hand. because I am lazy

In [1]:
import numpy as np
import torch
import random

import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch import load
from torch.nn import functional as F
from torch import autograd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#from ..easy_ntk import calculate_NTK
#from einops import rearrange

import time

import sys
from pathlib import Path

from numba import njit
from numba.typed import List

%matplotlib inline
%load_ext line_profiler
%load_ext memory_profiler

In [2]:
import copy

def _del_nested_attr(obj, names):
    """
    Deletes the attribute specified by the given list of names.
    For example, to delete the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'])
    """
    if len(names) == 1:
        delattr(obj, names[0])
    else:
        _del_nested_attr(getattr(obj, names[0]), names[1:])

def _set_nested_attr(obj, names, value):
    """
    Set the attribute specified by the given list of names to value.
    For example, to set the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'], value)
    """
    if len(names) == 1:
        setattr(obj, names[0], value)
    else:
        _set_nested_attr(getattr(obj, names[0]), names[1:], value)

def extract_weights(mod):
    """
    This function removes all the Parameters from the model and
    return them as a tuple as well as their original attribute names.
    The weights must be re-loaded with `load_weights` before the model
    can be used again.
    Note that this function modifies the model in place and after this
    call, mod.parameters() will be empty.
    """
    orig_params = tuple(mod.parameters())
    # Remove all the parameters in the model
    names = []
    for name, p in list(mod.named_parameters()):
        _del_nested_attr(mod, name.split("."))
        names.append(name)

    # Make params regular Tensors instead of nn.Parameter
    params = tuple(p.detach().requires_grad_() for p in orig_params)
    return params, names

def load_weights(mod, names, params):
    """
    Reload a set of weights so that `mod` can be used again to perform a forward pass.
    Note that the `params` are regular Tensors (that can have history) and so are left
    as Tensors. This means that mod.parameters() will still be empty after this call.
    """
    for name, p in zip(names, params):
        _set_nested_attr(mod, name.split("."), p)
        
def calculate_NTK(model,x,device='cpu',MODE='samples'):
    """
    INPUTS:
        model: torch.nn.Module 
        x: torch.Tensor
        device: 'cpu',
        MODE: 'minima'
    
    OUTPUTS:
        NTK: torch.Tensor
    
    Calculates the NTK for a model, p_dict a state dictionary, and x, a single tensor fed into the model
    
    The NTK is the grammian of the Jacobian of the model output to w.r.t. the weights of the model
    
    This function will output the NTK such that the minima matrix size is used. If the Jacobian is an NxM
    matrix, then the NTK is formulated so that if N < M; NTK is NxN. If M<N, then NTK is MxM.
    
    #EXAMPLE USAGE:
    device='cpu'
    model = MODEL() #a torch.nn.Module object 
    model.to(device)
    state_dict = model.state_dict()

    x_test = np.ones((100,1,28,28),dtype=np.float32)
    x_test = torch.from_numpy(x_test)

    NTK = calculate_NTK(model,x_test)
    """
    if not(MODE in ['minima','samples','params']):
        raise ValueError("MODE must be one of 'minima','samples','params'")
    
    x = x.to(device)
    x.requires_grad=False
    N = x.shape[0]
    M = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    #We need to create a clone of the model or else we make it unusable as part of the trickery 
    #to get pytorch to do what we want. Unforutantely, this exlcludes super big models. but, eh.
    model_clone = copy.deepcopy(model)
    
    params, names = extract_weights(model_clone)
    def model_ntk(*args,model=model_clone, names=names):
        params = tuple(args)
        load_weights(model, names, params)
        return model(x)
    
    Js = torch.autograd.functional.jacobian(model_ntk, tuple(params), create_graph=False, vectorize=True)
    
    Js = list(Js)
    #Js = [element for tupl in Js for element in tupl]
    #collapse the tensors
    for i,tensor in enumerate(Js):
        Js[i] = tensor.reshape(N,-1)
    
    J = torch.cat(Js,axis=1)
    
    if MODE=='minima':
        if N < M: #if datasize points is less than number of parameters:
            NTK = torch.matmul(J,J.T)

        if N >= M:#if number of parameters is less than datasize:
            NTK = torch.matmul(J.T,J)
    elif MODE=='samples':
        NTK = torch.matmul(J,J.T)
    elif MODE=='params':
        NTK = torch.matmul(J.T,J)
    
    return NTK

In [3]:
def relu(X,normalize=False):
    X = F.relu(X)
    if normalize:
        return np.sqrt(2*np.pi/(np.pi-1))*(X-1/np.sqrt(2*np.pi))
    else:
        return X
    

# #Identity
# def activation(x):
#     return x

# @njit
# def d_activation(x):
#     return np.ones(np.shape(x),dtype=np.float32) 


# #Tanh
def activation(x):
    return torch.tanh(x)

@njit
def d_activation(x):
    return np.cosh(x)**-2

In [4]:
SEED = 2

In [5]:
def NTK_weights(m):
    if isinstance(m, nn.Linear):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]
    if isinstance(m, nn.Conv2d):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]

In [6]:
# Easy NTK expects one output alone
class dumb_small(torch.nn.Module):
    '''
    simple network for test cases
    
    
    It seems like bias vectors aren't trivially added.
    '''
    def __init__(self,):
        super(dumb_small, self).__init__()
        
        self.d1 = torch.nn.Linear(2,3)
        self.d2 = torch.nn.Linear(3,3)
        self.d3 = torch.nn.Linear(3,1)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0))
        x_2 = activation(self.d2(x_1))
        x_3 = self.d3(x_2)
        return x_3
    
# Easy NTK expects one output alone
class dumb_small_layerwise(torch.nn.Module):
    '''
    simple network for test cases
    
    
    It seems like bias vectors aren't trivially added.
    '''
    def __init__(self,):
        super(dumb_small_layerwise, self).__init__()
        
        self.d1 = torch.nn.Linear(2,3)
        self.d2 = torch.nn.Linear(3,3)
        self.d3 = torch.nn.Linear(3,1)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0))
        x_2 = activation(self.d2(x_1))
        x_3 = self.d3(x_2)
        return x_3, x_2, x_1, x_0

In [7]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cpu'

model_small = dumb_small()
model_small.to(device)
model_small.apply(NTK_weights)

torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cpu'

model_layerwise = dumb_small_layerwise()
model_layerwise.to(device)
model_layerwise.apply(NTK_weights)

x_test = np.random.normal(0,1,(4,2)).astype(np.float32)
x_test = torch.from_numpy(x_test)

torch.Size([3, 2])
torch.Size([3, 3])
torch.Size([1, 3])
torch.Size([3, 2])
torch.Size([3, 3])
torch.Size([1, 3])


In [8]:
assert torch.all(model_layerwise.d1.weight == model_small.d1.weight)
assert torch.all(model_layerwise.d2.weight == model_small.d2.weight)
assert torch.all(model_layerwise.d3.weight == model_small.d3.weight)

assert torch.all(model_layerwise.d1.bias == model_small.d1.bias)
assert torch.all(model_layerwise.d2.bias == model_small.d2.bias)
assert torch.all(model_layerwise.d3.bias == model_small.d3.bias)

model_layerwise.d1.bias.detach().numpy().shape

(3,)

#### Jacobian autograd

In [9]:
NTK_easy = calculate_NTK(model_small,x_test).detach().numpy()

#### Can we get pytorch to give us the matrices with the correct kinds of derivative?

In [10]:
model_small.zero_grad()
x_4 = model_small(x_test)

In [11]:
#this method agrees between model layerwise and model small; meaning that the calculation is indepdent of those
#two models. the insinuation is somehting is wrong with both my methods for calculating,--- the same thing, since
#they agree with one another.

#in the future we would iterate over layers instead of like this...
layer_components_w1 = [] 
layer_components_w2 = []
layer_components_w3 = []

layer_components_b1 = []
layer_components_b2 = []
layer_components_b3 = []



for output in x_4:
    model_small.zero_grad()
    
    output.backward(retain_graph=True)

    #Get the tensors

    w3_grad = model_small.d3.weight.grad.detach().numpy()
    w2_grad = model_small.d2.weight.grad.detach().numpy()
    w1_grad = model_small.d1.weight.grad.detach().numpy()
    
    b3_grad = model_small.d3.bias.grad.detach().numpy()
    b2_grad = model_small.d2.bias.grad.detach().numpy()
    b1_grad = model_small.d1.bias.grad.detach().numpy()


    #reshape and append. deep copy neccessary or else they are the same objects
    layer_components_w1.append(w1_grad.reshape(-1).copy())
    layer_components_w2.append(w2_grad.reshape(-1).copy())
    layer_components_w3.append(w3_grad.reshape(-1).copy())

    layer_components_b1.append(b1_grad.reshape(-1).copy())
    layer_components_b2.append(b2_grad.reshape(-1).copy())
    layer_components_b3.append(b3_grad.reshape(-1).copy())


In [12]:
layer_components_b3

[array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32)]

In [13]:
layer_components_w1 = np.array(layer_components_w1)
layer_components_w2 = np.array(layer_components_w2)
layer_components_w3 = np.array(layer_components_w3)

layer_components_b1 = np.array(layer_components_b1)
layer_components_b2 = np.array(layer_components_b2)
layer_components_b3 = np.array(layer_components_b3)

In [14]:
autograd_NTK = layer_components_w1 @ layer_components_w1.T+\
    layer_components_w2 @ layer_components_w2.T+\
    layer_components_w3 @ layer_components_w3.T+\
    layer_components_b1 @ layer_components_b1.T+\
    layer_components_b2 @ layer_components_b2.T+\
    layer_components_b3 @ layer_components_b3.T

# Layerwise exact

In [88]:
@njit #no parallel transformation available ;#fasterer
def cross(X):
    return X.T.dot(X)

def compute_NTK_new_WithBias(Ws, Bs, Xs, d_int, d_array):
    L = len(Xs)-1 #number of layers, Xs goes from inputs to right before outputs; X_0 is the input, X_L CK
    #components = []
    n = Xs[0].shape[1] #number of datapoints

    #holds the derivatives, first value is empty list...?; just a spacer, replace with array
    Ds = [np.array([[0.0]],dtype=np.float32)] 
    for l in range(L):
        Ds.append(d_activation(np.dot(Ws[l],Xs[l]) + Bs[l]))
    #The first term is just conjugate kernel
    KNTK = cross(Xs[L])
    #components.append(cross(Xs[L]))
    ####################
    #Ds must be right, because this block is creating correct.
    for l in range(1,L+1):#l counts layers going forward from 1...
        #we are going to construct terms that look like ( S^T S ) * (X^T X)
        XtX = cross(Xs[l-1])
        S = np.expand_dims(Ws[-1].T.reshape(-1)/np.sqrt(d_array[L]),axis=1) #has shape input to last layer.
        for k in range(L,l-1,-1): #counts backwards from l
            S = Ds[k]*S
            if k > l:
                S = np.dot(S.T,Ws[k-1]).T/np.sqrt(d_array[k-1])
        #components.append(cross(S) * XtX)
        KNTK += cross(S) * XtX
    ####################
    for l in range(L+1,0,-1):
        S=1
        for k in range(L,l-1,-1):
            
            S = np.dot(Ws[k].T/np.sqrt(d_array[k]),S)
            
            S = S * Ds[k]

        S = np.multiply(np.ones((d_int[l],n),dtype=np.float32),S)
        #components.append(cross(S))
        KNTK += cross(S)
    return KNTK#, components

In [89]:
x_3, x_2, x_1, x_0 = model_layerwise(x_test)

Ws = []
Ws.append(model_layerwise.d1.weight.detach().numpy().astype(np.float32))
Ws.append(model_layerwise.d2.weight.detach().numpy().astype(np.float32))
Ws.append(model_layerwise.d3.weight.detach().numpy().astype(np.float32))

Xs = [] # Xs are shape (output x #DP) ; however, typical python notation is reversed, so we take transpose here
Xs.append(x_0.detach().numpy().T.astype(np.float32))
Xs.append(x_1.detach().numpy().T.astype(np.float32))
Xs.append(x_2.detach().numpy().T.astype(np.float32))

Bs = []
Bs.append(model_layerwise.d1.bias.detach().numpy().astype(np.float32)[:,None])
Bs.append(model_layerwise.d2.bias.detach().numpy().astype(np.float32)[:,None])
Bs.append(model_layerwise.d3.bias.detach().numpy().astype(np.float32)[:,None])

ds_int = []
ds_int.append(0)
ds_int.append(3)
ds_int.append(3)
ds_int.append(1)

ds_array = []
ds_array.append(np.array([0.0],dtype=np.float32)) #first element is the input length
ds_array.append(np.array([1.0],dtype=np.float32))
ds_array.append(np.array([1.0],dtype=np.float32)) #the remaining elements are the output lengths, but omit the last output length assumed 1.
ds_array.append(np.array([1.0],dtype=np.float32))

In [90]:
layerwise_NTK, components = compute_NTK_new_WithBias(Ws, Bs, Xs, ds_int, ds_array)

In [91]:
layer_components_b2

array([[-3.6603562e-03,  9.8079455e-01, -1.1442316e-01],
       [-1.4193730e-03,  2.4364762e-01, -2.2082861e-01],
       [-9.4461028e-04,  4.8596486e-02, -2.6164386e-01],
       [-6.7316373e-03,  1.0789857e+00, -1.1431656e-01]], dtype=float32)

# Compare all against each other

In [92]:
layerwise_NTK

array([[ 8.254051 ,  4.5994496,  2.5208883,  7.149842 ],
       [ 4.5994496,  5.586909 ,  3.136908 ,  1.288177 ],
       [ 2.5208883,  3.136908 ,  3.7187269,  1.900444 ],
       [ 7.149842 ,  1.288177 ,  1.900444 , 12.942534 ]], dtype=float32)

In [93]:
NTK_easy

array([[ 8.254052 ,  4.59945  ,  2.5208886,  7.149842 ],
       [ 4.59945  ,  5.586909 ,  3.1369083,  1.2881768],
       [ 2.5208886,  3.1369083,  3.7187266,  1.9004439],
       [ 7.149842 ,  1.2881768,  1.9004439, 12.942536 ]], dtype=float32)

In [94]:
autograd_NTK

array([[ 8.254051 ,  4.59945  ,  2.5208883,  7.1498413],
       [ 4.59945  ,  5.5869093,  3.136908 ,  1.2881768],
       [ 2.5208883,  3.136908 ,  3.7187269,  1.9004438],
       [ 7.1498413,  1.2881768,  1.9004438, 12.942534 ]], dtype=float32)

# Compare components against one another

In [95]:
#Since this doesnt depend on the X
print(layer_components_w1 @ layer_components_w1.T)
print(layer_components_w2 @ layer_components_w2.T)
print(layer_components_w3 @ layer_components_w3.T)

print(layer_components_b1 @ layer_components_b1.T)
print(layer_components_b2 @ layer_components_b2.T)
print(layer_components_b3 @ layer_components_b3.T)

[[ 0.53224283  0.56983453  0.2124167  -0.40349403]
 [ 0.56983453  1.8930786   0.2896796  -2.2823007 ]
 [ 0.2124167   0.2896796   0.22192383  0.04162453]
 [-0.40349403 -2.2823007   0.04162453  5.0680943 ]]
[[0.96924645 0.2718444  0.07023415 0.973709  ]
 [0.2718444  0.17904218 0.11069712 0.17418593]
 [0.07023415 0.11069712 0.16018681 0.04559045]
 [0.973709   0.17418593 0.04559045 1.2190182 ]]
[[1.7679785  1.7794366  0.89337176 1.6160638 ]
 [1.7794366  2.1456802  1.5486958  1.3759024 ]
 [0.89337176 1.5486958  2.2092543  0.4464258 ]
 [1.6160638  1.3759024  0.4464258  1.6681615 ]]
[[3.00952    0.7140929  0.26726115 2.8921945 ]
 [0.7140929  0.2609767  0.11821551 0.7322429 ]
 [0.26726115 0.11821551 0.05654183 0.2844516 ]
 [2.8921945  0.7322429  0.2844516  2.8099377 ]]
[[0.975064   0.26424137 0.07760475 1.0713685 ]
 [0.26424137 0.10813145 0.06962021 0.2881462 ]
 [0.07760475 0.06962021 0.07082002 0.0823515 ]
 [1.0713685  0.2881462  0.0823515  1.1773237 ]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1

In [97]:
print(components[1])
print(components[2])
print(components[0])

print(components[5])
print(components[4])
print(components[3])

[[ 0.5322428   0.5698344   0.21241668 -0.40349403]
 [ 0.5698344   1.8930782   0.28967956 -2.2823005 ]
 [ 0.21241668  0.28967956  0.2219238   0.04162453]
 [-0.40349403 -2.2823005   0.04162453  5.0680943 ]]
[[0.96924627 0.27184442 0.07023415 0.97370887]
 [0.27184442 0.1790422  0.11069711 0.17418592]
 [0.07023415 0.11069711 0.16018678 0.04559045]
 [0.97370887 0.17418592 0.04559045 1.2190182 ]]
[[1.7679785  1.7794366  0.89337176 1.6160638 ]
 [1.7794366  2.1456802  1.5486958  1.3759024 ]
 [0.89337176 1.5486958  2.2092543  0.4464258 ]
 [1.6160638  1.3759024  0.4464258  1.6681615 ]]
[[3.00951977 0.71409287 0.2672611  2.89219451]
 [0.71409287 0.26097667 0.1182155  0.73224284]
 [0.2672611  0.1182155  0.05654182 0.28445158]
 [2.89219451 0.73224284 0.28445158 2.80993776]]
[[0.97506386 0.26424137 0.07760474 1.07136834]
 [0.26424137 0.10813146 0.0696202  0.28814626]
 [0.07760474 0.0696202  0.07082    0.08235151]
 [1.07136834 0.28814626 0.08235151 1.17732378]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1

# huh

In [100]:
print(np.allclose(components[1],layer_components_w1 @ layer_components_w1.T))
print(np.allclose(components[2],layer_components_w2 @ layer_components_w2.T))
print(np.allclose(components[0],layer_components_w3 @ layer_components_w3.T))
print(np.allclose(components[5],layer_components_b1 @ layer_components_b1.T))
print(np.allclose(components[4],layer_components_b2 @ layer_components_b2.T))
print(np.allclose(components[3],layer_components_b3 @ layer_components_b3.T))

True
True
True
True
True
True
